In [20]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad.
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [21]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/learning-datasets/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  # Your Code
  def epochi_end(self,epochs,logs={}):
    if logs.get('accuracy') >= DESIRED_ACCURACY:
      print("\nReached 99.9% accuracy on training bratan\n")
      self.model.stop_training = True

callbacks = myCallback()

--2025-02-11 18:00:45--  https://storage.googleapis.com/learning-datasets/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.207, 173.194.210.207, 173.194.212.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2025-02-11 18:00:45 (144 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [22]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
    # This is the first convolution
    tf.keras.layers.Conv2D(35, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(27, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(500, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 148, 148, 35)        │             980 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 74, 74, 35)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 72, 72, 27)          │           8,532 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 36, 36, 27)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 34992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 500)                 │      17,496,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             501 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,506,513 (66.78 MB)

 Trainable params: 17,506,513 (66.78 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# This code block should create an instance of an ImageDataGenerator called train_datagen
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [24]:
# This code block should call model.fit and train for
# a number of epochs.
history = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=30,
      callbacks=[callbacks])

# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5000 - loss: 0.6923
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5000 - loss: 7.3272
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5000 - loss: 6.1036
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5000 - loss: 0.6762
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5000 - loss: 0.6100
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5000 - loss: 0.6167
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5000 - loss: 0.6739
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6125 - loss: 0.5645
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9375 - loss: 0.4269
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9500 - loss: 0.3099
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9375 - loss: 0.2317
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 1.0000 - loss: 0.2010
Epoch 13/30
8